## Creating isolated claude code workspaces

Using claude in claude code cli, you do not get to specify the finegrained specification for each Task agent, and a lot of times you will need to use `--dangerously-skip-permissions` to run the code. This is not ideal for production use.

### 1. Understanding Claude Code configuration

- claude code runs in a given workspace (directory), the essential settings is kept at `workspace/.claude/settings.json`
- claude code also reads the `workspace/CLAUDE.md` as system prompt

These configuration dictates how claude code will behave, 

In [3]:
agent_config = ".khive/agents/analyst"
REPO = "/Users/lion/projects/khivedev"
CC_WORKSPACE = ".khive/workspaces"
FLOW_NAME = "memory_mcp_flow0"


from pathlib import Path
from typing import Literal
from lionagi.utils import create_path
from lionagi import iModel

AgentRole = Literal[
    "orchestrator",
    "analyst",
    "architect",
    "auditor",
    "commentator",
    "critic",
    "implementer",
    "innovator",
    "researcher",
    "reviewer",
    "strategist",
    "tester",
    "theorist",
]


def create_cc(
    flow_name: str,
    role: AgentRole,
    agent_suffix: str = "",
    model: str = "sonnet",
    verbose_output: bool = True,
    auto_finish: bool = False,
):
    """Create Claude Code model instance for LionAGI operations."""

    params = {"permission_mode": "acceptEdits"}
    if role == "orchestrator":
        params["permission_mode"] = "bypassPermissions"

    elif role not in ["tester", "reviewer", "architect", "implementer"]:
        params["ws"] = f"{CC_WORKSPACE}/{flow_name}/{role}{agent_suffix}"
        params["add_dir"] = "../../../../"
        text = Path(
            f"{REPO}/{CC_WORKSPACE}/agents/{role}/.claude/settings.json"
        ).read_text()
        fp = create_path(
            directory=f"{REPO}/{CC_WORKSPACE}/{flow_name}/{role}{agent_suffix}/.claude",
            filename="settings.json",
            file_exist_ok=True,
        )
        fp.write_text(text)

        # text = Path(
        #     f"{REPO}/{CC_WORKSPACE}/agents/{role}/.claude/.mcp.json"
        # ).read_text()
        # fp = create_path(
        #     directory=f"{REPO}/{CC_WORKSPACE}/{flow_name}/{role}{agent_suffix}",
        #     filename=".mcp.json",
        #     file_exist_ok=True,
        # )
        # fp.write_text(text)

    return iModel(
        provider="claude_code",
        endpoint="query_cli",
        model=model,
        api_key="dummy_api_key",
        verbose_output=verbose_output,
        cli_display_theme="dark",
        auto_finish=auto_finish,
        repo=REPO,
        **params,
    )

In [4]:
cc = create_cc(FLOW_NAME, role="analyst")